In [1]:
from threading import Thread
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer

from trl import (
    ModelConfig,
    RewardConfig,
    RewardTrainer,
    ScriptArguments,
    get_kbit_device_map,
    get_peft_config,
    get_quantization_config,
    setup_chat_format,
)

from transformers import DataCollatorWithPadding
from typing import List, Dict, Any
from transformers import PreTrainedTokenizerBase

/home/hank/projects/polycrest-llm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-21 09:03:12,092] [INFO] [real_accelerator.py:222:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlvsym'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlopen'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlclose'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlerror'
/usr/bin/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `dlsym'
collect2: error: ld returned 1 exit status


In [2]:
from datasets import load_dataset
ds = load_dataset("Hankbeasley/polycodertext")['train']


max_length = max(len(ids) for ids in ds['accept'])
print(f"Maximum length of prompt_input_ids: {max_length}")



Maximum length of prompt_input_ids: 83255


In [3]:
tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
model = AutoModelForCausalLM.from_pretrained(
        "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B",
        device_map="auto",  
        torch_dtype=torch.bfloat16,
        attn_implementation="flash_attention_2"
    )


In [4]:
input = tokenizer(ds['accept'][0], return_tensors="pt")
with torch.no_grad():
    decoded = model(input['input_ids'])
predicted_ids = torch.argmax(decoded.logits[0], dim=-1)
output_text = tokenizer.decode(predicted_ids, skip_special_tokens=True)
#input_text = tokenizer.decode(inputIds, skip_special_tokens=True)
print(input)
print(output_text)
# Clean up tensors to free memory
del decoded
del predicted_ids
torch.cuda.empty_cache()  # If using GPU

{'input_ids': tensor([[151646,  30721,  54615,  ...,     23,  25439,   3417]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1]])}
)

)

 theYou {u>
  The are a data writer AI, to help with with detailed-quality, detailed, and safe-ocumented code. Your will access to a programming and generate in your generation and such, and optimization.    <Overview>

  <Prompt>
    Code 1 a, detailed code to for to the problem's problem.
>
>
    <Question>Use the practices practices.
 conventions appropriate, concise explanations.
 your code.
Instruction>
    <Instruction>Includeplain your performance thoroughly any in.
 providing.</Instruction>
</ <Instruction>etsage the tools to optimize and debug, and optimize your.</ applicable.</Instruction>
</ </Instructions>
> <Problem>
    You capability>Access code for multiple programming languages (Capability>
    <Capability>Opt solutions explanations and analysis</ the solutions</Capability>
    <Capability>Anist with code and performance analysis</

In [4]:
def preprocess_function(examples):
    parts = examples['accept'].split('<think>')
    if (len(parts) < 2 or len(parts) > 2):
        return None
    assistantparts = examples['accept'].split("<｜Assistant｜>")
    if (len(assistantparts) > 2):
        return None
    examples['prompt'] = parts[0]
    examples['chosen'] = parts[1]
    partsrejected = examples['reject'].split('<think>')
    examples['rejected'] = "<think>" + partsrejected[1]
    return examples

ds = ds['train'].map(preprocess_function)
#print(ds)
formated = ds.remove_columns(['accept', 'reject', 'testname'])
print(formated)

Map: 100%|██████████| 10471/10471 [00:01<00:00, 5461.94 examples/s]

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 8938
})


In [42]:

#print(formated[0]['prompt'])
#print(formated)
parts = ds[0]['accept'].split('<think>')
prompt = ds[0]['accept'].split('<think>')[0]
#print(parts[1])

tokenizer = AutoTokenizer.from_pretrained("deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B")
tokenized = tokenizer(formated[0]['rejected'], add_special_tokens=True)
print(tokenized["input_ids"])
#print(tokenizer.convert_ids_to_tokens(tokenized["input_ids"]))
#print(tokenizer.eos_token)


[151646, 151648, 198, 71486, 11, 358, 2776, 16601, 448, 21828, 458, 64595, 309, 24279, 24626, 1667, 12914, 13, 576, 5795, 374, 311, 3270, 264, 729, 429, 646, 1477, 279, 4396, 15723, 31172, 369, 1817, 6524, 1741, 429, 279, 2661, 23606, 9982, 830, 13, 6771, 752, 1438, 1495, 1246, 358, 646, 5486, 419, 382, 5338, 11, 358, 1184, 311, 3535, 279, 3491, 17247, 13, 1674, 759, 309, 24279, 46523, 8290, 1817, 6524, 448, 264, 4911, 15723, 11, 323, 279, 6388, 11931, 4157, 387, 7168, 13, 576, 3110, 2661, 374, 55663, 488, 18447, 284, 93462, 11, 892, 46918, 311, 220, 24, 20, 21, 22, 488, 220, 16, 15, 23, 20, 284, 220, 16, 15, 21, 20, 17, 13, 8886, 6524, 33210, 311, 264, 2155, 15723, 11, 323, 279, 2629, 1969, 387, 4396, 382, 4416, 11, 279, 729, 358, 2776, 311, 3270, 11, 11625, 1507, 3880, 311, 1896, 458, 23606, 438, 1946, 323, 470, 458, 1633, 12731, 1817, 6524, 311, 1181, 12159, 15723, 382, 40, 3278, 1191, 553, 22314, 279, 1946, 23606, 13, 576, 1946, 686, 387, 264, 914, 304, 279, 3561, 330, 61674, 488, 